In [24]:
import os

# os.chdir(r"C:\Users\HP\Desktop\face_emotion_github\Face-Emotion-Recognition-main")
print(os.getcwd())

c:\Users\HP\Desktop\face_emotion_github\Face-Emotion-Recognition-transferlearning


In [25]:
#Import ML required packages
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd 
import keras
from keras.utils import to_categorical
#Import Libraries before model creation
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
import tensorflow as tf
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [27]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

 /device:CPU:0 || Unnamed device || CPU || 256.0 MiB
 /device:GPU:0 ||  NVIDIA GeForce RTX 3050 Laptop GPU || GPU || 1.6 GiB


#### LOAD DATASET

In [47]:
# Define data augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load and preprocess the training and validation data
train_generator = datagen.flow_from_directory(
    '../Data/Fer2013/archive/train/',
    target_size=(48, 48),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    classes=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    '../Data/Fer2013/archive/test/',
    target_size=(48, 48),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    classes=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
    shuffle=True
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


### Original VGG-16 & Transfer Learning

In [48]:
# Load the VGG16 model without the top layer
base_model = VGG16(include_top=False, input_shape=(48, 48, 3))

In [49]:
# Freeze the pre-trained layers in the base model
for layer in base_model.layers:
    layer.trainable = False


In [50]:
# Add custom classification layers on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x)

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0   

c:\Users\HP\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [51]:
# Train the model
# model.fit(train_generator, epochs=50, validation_data=val_generator)
model.fit(train_generator, epochs=50, validation_data=val_generator)

Epoch 1/50
898/898 [==============================] - 76s 78ms/step - loss: 1.7510 - accuracy: 0.2880 - val_loss: 1.6924 - val_accuracy: 0.3260
Epoch 2/50
898/898 [==============================] - 51s 57ms/step - loss: 1.7036 - accuracy: 0.3207 - val_loss: 1.6593 - val_accuracy: 0.3501
Epoch 3/50
898/898 [==============================] - 48s 53ms/step - loss: 1.6876 - accuracy: 0.3251 - val_loss: 1.6500 - val_accuracy: 0.3458
Epoch 4/50
898/898 [==============================] - 49s 54ms/step - loss: 1.6756 - accuracy: 0.3353 - val_loss: 1.6479 - val_accuracy: 0.3502
Epoch 5/50
898/898 [==============================] - 51s 57ms/step - loss: 1.6728 - accuracy: 0.3354 - val_loss: 1.6321 - val_accuracy: 0.3558
Epoch 6/50
898/898 [==============================] - 50s 55ms/step - loss: 1.6693 - accuracy: 0.3366 - val_loss: 1.6399 - val_accuracy: 0.3508
Epoch 7/50
898/898 [==============================] - 49s 55ms/step - loss: 1.6611 - accuracy: 0.3433 - val_loss: 1.6362 - val_accuracy:

In [56]:
eval = model.evaluate_generator(val_generator)
print(eval)

C:\Users\HP\AppData\Local\Temp\ipykernel_7448\1447940332.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  eval = model.evaluate_generator(val_generator)


[1.6024103164672852, 0.38548341393470764]


In [57]:
eval_metrics = model.evaluate_generator(val_generator)
test_acc = eval_metrics[1]
print("Test accuracy:", test_acc)

C:\Users\HP\AppData\Local\Temp\ipykernel_7448\557385363.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  eval_metrics = model.evaluate_generator(val_generator)


Test accuracy: 0.38186123967170715


In [53]:
import json
model_json = model.to_json()
with open("../Models/transferlearning/onlytransfer/model.json", "w") as json_file:
  json_file.write(model_json)
  model.save_weights("../Models/transferlearning/onlytransfer/model.h5")
print("Saved model to disk")

Saved model to disk
